# Project Description:

In this project the salary increasement of MBA players between 1985 and 2015 is analyzed. 

For this analysis a [data set](http://www.seanlahman.com/baseball-archive/statistics/) provided by SeanLahman.com is used. It contains statistics related to the MBA from year 1871 till 2015.
We use the salary related part of the statistics in the dataset that start in the year 1985, as basis for your evaluation.

## Point of Analyses

The analyses in this project concentrates on these questions: 
* How did the salaraies for MBA player changed from the year 1985 and 2015 in generall 
* Which players main positon gained the most salaries
* How much salary does the MVP player become in relation to the avg. salary player

# Project Preparation

## Import Libaries

In [1]:
import os
import shutil
import io
import urllib.request
import zipfile

from IPython.display import display

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

## Set Global Variables

In [2]:
URL = 'http://seanlahman.com/files/database/baseballdatabank-master_2016-03-02.zip'
PATH_DATASET = 'data/'

## Download the data

In [3]:
def download_unpack_zip_folder(url, path_data):
    """Download a csv file and stores it in the data folder of the project repository.

    Args:
        URL of the csv file

    Returns:
        None
    """
    ### Set file name for cached zip file
    cached_zip = 'data/temp.zip'
    ### Set path to data folder
    path_dir = path_data
    
    ### Create the data folder if not exits
    if not os.path.exists(path_dir):
        os.makedirs(path_dir)
    
    ### fetch the zip file
    file = urllib.request.urlopen(url)
    
    ### Cache the zip file on the disk
    output = open(cached_zip, 'wb')
    output.write(file.read())
    output.close()
    
    ### Unpack the zip file
    zfile = zipfile.ZipFile(cached_zip)
    zfile.extractall(path_dir)
    zfile.close()
    
    ### Delete the zip file
    os.remove(cached_zip)
    
    ### Initalize list of files in the zip content folder
    file_names = []
    ### Get all file names inside the unpacked zipfile
    for subdir, dirs, files in os.walk(path_dir):
        for file in files:
            file_names.append(os.path.join(subdir, file))
    
    ### Get all files with the extension .csv
    file_names = [file for file in file_names if '.csv' in file]
    
    ### Move csv files from zip content folder to data folder
    for file in file_names:
        new_file = path_dir + file.split('\\')[-1]
        os.rename(file, new_file)
    
    ### Delete original zip content folder
    shutil.rmtree(file_names[0].split('\\')[0])

"""
Downloads the data to the data folder of a local repository after you run it once you can uncomment this lines.
To prevent the code from downloading the data every time you run the code.
"""
download_unpack_zip_folder(URL, PATH_DATASET)

FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'data/AllstarFull.csv' -> 'data/data/AllstarFull.csv'

## Load all csv files of a folder into a dict of dfs

In [ ]:
def get_file_names(path):
    """Gets an folder path and returns names of all csv files contained in the folder

    Args:
        Path to folder

    Returns:
        List of file names
    """
    file_names = os.listdir(path)
    
    file_names = [file for file in file_names if '.csv' in file]
    
    return file_names
    
def load_csv(path):
    """Get the path to a csv file and returns a dataframe

    Args:
        Path of the csv file

    Returns:
        Dataframe of given csv file
    """
    df = pd.read_csv(path)
    
    return df

def load_csv_from_folder(path):
    """Gets an folder path and returns a dict containing dataframes for all csv files in the folder

    Args:
        URL of folder containing csv files

    Returns:
        Dict of dataframes
    """
    file_names = get_file_names(path)
    
    data =  {}
    
    for file in file_names:
        
        data[file] = load_csv(path + file)
    
    return data

## Load the Dataset

In [ ]:
def print_heads_of_dict_of_dfs(data):
    """Gets a dict of dataframes and prints the first 5 entries of every dataframe

    Args:
        Dict of dataframes

    Returns:
        None
    """
    for key, df in data.items():
        print(key)
        display(df.head())
        print('----------------------------------------')

In [ ]:
# Load the data as dfs in the dict data
data = load_csv_from_folder(PATH_DATASET)
# Inspect the first 5 rows of the dict of dataframes
print_heads_of_dict_of_dfs(data)

# Evaluation

## Evaluate avg. Salaries per year

### Analyse the Salaries data set

In [ ]:
### Analyse the count of entries in the data set
print('len: ' + str(data['Salaries.csv'].shape[0]))

In [ ]:
### Analyse the point estimators of the data set
print(data['Salaries.csv']['salary'].describe())

The summary shows that the dataset contains 25575 entries, with an:
* average salary of 2,008,653
* min salary of 0
* max salary of 33,000,000
* medain salary of 550,000

In [ ]:
### Analyse the values for year in the data set
print(data['Salaries.csv']['yearID'].unique())

The Salaries.csv dataset contains 25575 entries. For the years 1985 to 2015.

### Remove entries in the Salaries data set that contain a salary of zero

In [ ]:
# Filter all salaries that are 0 or less
df_salaries_cleaned = data['Salaries.csv'][data['Salaries.csv']['salary'] >0.0]
### Analyse the count of remaining entries in the data set
print('len: ' + str(df_salaries_cleaned.shape[0]))

Filter out all the entries in the Salaries.csv file with a salary of 0 or less, because this indicates either a false value or that the player never activaly played.
This process reduced the number of entries fro 25575 to 25573

### Summarize salaries per year

In [ ]:
### Group salaries per year
salary_per_year = df_salaries_cleaned.groupby(['yearID'],as_index=False).mean()
### Reset index to year
salary_per_year.set_index(['yearID'], inplace=True)
### Inspect the dataset
display(salary_per_year.transpose())

### Chart salaries per year

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(12)

ax.plot(salary_per_year.index, salary_per_year['salary'], color='black')

ax.set_title('Salaries per Year')
ax.set_xlabel('Year')
ax.set_ylabel('Salary')
ax.set_facecolor('white')
ax.set_xticks(salary_per_year.index)
ax.set_xticklabels(salary_per_year.index, rotation=45)

plt.show()

As you can see in the chart the salary drasticly improved over the last 30 years.
From about 50,000 avg. salary in the year between 1985 to 1990 to over 4,000,000 since 2014.

### Compute salaries increase per year

In [ ]:
### Generate a new column growth in salary between years
salary_per_year["salary_increase"] = np.nan

### Compute the salaray growth
for key, _ in salary_per_year.iterrows():
    
    if key - 1 in salary_per_year.index:
        increase = (salary_per_year.ix[key, 'salary'] / (salary_per_year.ix[key - 1, 'salary'] / 100)) - 100
        salary_per_year.ix[key, 'salary_increase'] = increase
    else:
        salary_per_year.ix[key, 'salary_increase'] = 0
        
### Inspect the dataset
display(salary_per_year.transpose())

### Chart salaries increase per year

In [ ]:
bar_heights = salary_per_year["salary_increase"].values
bar_positions = np.arange(len(salary_per_year.index))

fig, ax = plt.subplots()
fig.set_figwidth(18)
fig.set_figheight(6)

ax.bar(bar_positions, bar_heights, .5, color='silver')
ax.set_title('Salary increase per year')
ax.set_xlabel('Year')
ax.set_ylabel('Salary increase to the year before in %')

ax.set_xticks(np.arange(len(salary_per_year.index)))
ax.set_xticklabels(salary_per_year.index)

ax.set_ylim([-15,75])
ax.axhline(bar_heights.mean(), color='black', linestyle='dashed', linewidth=1)
line_text = 'Avg. increase (' + str(round(bar_heights.mean(),2)) + '%)'
ax.text(0, bar_heights.mean() + 0.5 , line_text, color='black')
ax.set_facecolor('white')
plt.show()

As the chart shows the average salaray increase is about 8%. And the highest increase in salary was from the year 1990 to the year 1991.

## Evaluate avg. Salaries per position

As the dataset Salaries.csv contains no player postion values it becomes merged with the Fielding.csv dataset that contains values for the player position.  
In doing so in the Fielding.csv dataset for every player the position he played most in one year is evaluted and the salary is used as his main position.
Later the Fielding.csv dataset is merged with the Salaries.csv dataset based on playerID, yearID, teamID.
And group by the position, to evalute the average salary for every possible main position.

### Prepare the Salaries df for merging with the Fielding df

In [ ]:
### Copy df in a new var    
df_salaries = data['Salaries.csv'].copy()
### Convert column yearID to string
df_salaries['yearID'] =df_salaries['yearID'].astype('int')
### Sort the df
df_salaries.sort_index(inplace=True)
### Remove any unnessary columns
df_salaries.drop(['lgID'], inplace=True, axis=1)

### Inspect the dataset
display(df_salaries.head())

### Prepare the Fielding df for merging with the Salaries df

In [ ]:
### Copy df in a new var
df_fielding = data['Fielding.csv'].copy()
### Convert column yearID to string
df_fielding['yearID'] =df_fielding['yearID'].astype('int')
### Remove any entries < 1985
df_fielding = df_fielding[df_fielding['yearID'] >= 1985]
### Generate a pseudo column consiting of player and year values
df_fielding['player_year'] = df_fielding['playerID'] + ' ' + str(df_fielding['yearID'])
### Remove any duplicates in the pseudo column player_year and keep just the column with the most game appearances
df_fielding = df_fielding.sort_values(by=['G']).drop_duplicates(subset='player_year', keep='last')
### Sort the df
df_fielding.sort_index(inplace=True)
### Remove any unnessary columns
df_fielding = df_fielding[['playerID', 'yearID', 'teamID', 'POS']]

### Inspect the dataset
display(df_fielding.head())

### Merge the two dfs

In [ ]:
### Merge the dfs based on the columns playerID, yearID, teamID
df_merged = df_salaries.merge(df_fielding, how='right', on=['playerID', 'yearID', 'teamID'])

### Inspect the dataset
display(df_merged.head())

### Summarize salary per position

In [ ]:
### Summarize the salary py position
df_salary_per_position = df_merged.groupby(['POS'],as_index=False).mean()
### Remove any unnessary columns
df_salary_per_position.drop(['yearID'], inplace=True, axis=1)
### Reset index to positon
df_salary_per_position.set_index(['POS'], inplace=True)

### Inspect the dataset
display(df_salary_per_position.transpose())

### Chart avg. salaries per main position

In [ ]:
bar_heights = df_salary_per_position['salary'].values
bar_positions = np.arange(df_salary_per_position.shape[0]) + 0.75

fig, ax = plt.subplots()
fig.set_figwidth(12)
fig.set_figheight(8)

ax.bar(bar_positions, bar_heights, .5, color='silver')
ax.set_title('Avg. Salary per Position')
ax.set_xlabel('Position')
ax.set_ylabel('Salary')
ax.set_ylim([0,5000000])
ax.set_facecolor('white')
ax.set_xticks(bar_positions)
ax.set_xticklabels(df_salary_per_position.index, rotation=90)

plt.show()

The chart shows that over the past 30 years the players that mostly play as LF gets in average the most salary and the players that mostly play as DH gets in average the lowest salary.

## Evaluate the difference between a avg. Salaries per year with the salary of the MVP in this year

To evaluate the diffence the dataset AwardsPlayers.csv is filtered for all players that became the MVP award and years later than 1984. Than the avg. salaries per year are computed by group the df_salaries_cleaned dataframe per year. Later the df_salaries_cleaned and the df_mvp_salary dataframe are merged based on the year and the difference between this two salary values is computed. The difference is computed in percent by using the avg. salary as basis. So the differnce is represented as difference to the avg. salary.

### Filter the AwardsPlayers.csv dataframe and prepare for the merge

In [ ]:
### Filter the AwardsPlayers.csv for player award MVP(TSN Major League Player of the Year) and years later than 1984
df_mvp_1985_till_2015 = data['AwardsPlayers.csv'][(data['AwardsPlayers.csv']['awardID'] == \
                                                   'TSN Major League Player of the Year') \
                                                  & (data['AwardsPlayers.csv']['yearID'] >= \
                                                     1985)][['playerID', 'yearID']]

### Select the columns playerID, yearID
df_mvp_1985_till_2015 = df_mvp_1985_till_2015[['playerID', 'yearID']]

### Merge df_mvp_1985_till_2015 with salaries basesd on playerID, yearID
df_mvp_salary = df_mvp_1985_till_2015.merge(df_salaries, how='left', on=['playerID', 'yearID'])

### Drop the teamID, playerIDcolumn because not needed
df_mvp_salary.drop(['teamID', 'playerID'], inplace=True, axis=1)

### Set year column as index
df_mvp_salary.set_index('yearID', inplace = True)

### Rename salary column to avoid the same column name
df_mvp_salary.rename(columns= {'salary': 'mvp_salary'}, inplace=True)

### Inspect df_mvp_salary
display(df_mvp_salary.transpose())

### Summarize salaries per year

In [ ]:
### Group salaries per year
salary_per_year = df_salaries_cleaned.groupby(['yearID'],as_index=False).mean()

### Reset index to year
salary_per_year.set_index(['yearID'], inplace=True)

### Rename salary column to avoid the same column name
salary_per_year.rename(columns= {'salary': 'avg_salary'}, inplace=True)

### Inspect the dataset
display(salary_per_year.transpose())

### Merge df_mvp_salary with salary_per_year

In [ ]:
### Merge df_mvp_salary with salary_per_year basesd on playerID
df_mvp_avg_salary = df_mvp_salary.merge(salary_per_year, how='left', left_index=True, right_index=True)

### Inspect the dataset
display(df_mvp_avg_salary.transpose())

### Compute the difference in % between the MVP and the avg. player

In [ ]:
### Compute avg. difference between mvp salary in percent
df_mvp_avg_salary['difference'] = (df_mvp_avg_salary['mvp_salary'] / df_mvp_avg_salary['avg_salary'] * 100) - 100

### Inspect the dataset
display(df_mvp_avg_salary.transpose())
print(df_mvp_avg_salary['difference'].mean())


### Chart difference in salary between the avg. salary of the players and the MVP

In [ ]:
bar_heights = df_mvp_avg_salary['difference'].values
bar_positions = np.arange(len(df_mvp_avg_salary.index))

fig, ax = plt.subplots()
fig.set_figwidth(18)
fig.set_figheight(6)

ax.bar(bar_positions, bar_heights, .5, color='silver')
ax.set_title('Salary difference MVPs and Avg. salary')
ax.set_xlabel('Year')
ax.set_ylabel('Salary difference in %')

ax.set_xticks(np.arange(len(df_mvp_avg_salary.index)))
ax.set_xticklabels(df_mvp_avg_salary.index, rotation=45)

ax.axhline(bar_heights.mean(), color='red', linestyle='dashed', linewidth=1)
line_text = 'Avg. increase (' + str(round(bar_heights.mean(),2)) + '%)'
ax.text(0, bar_heights.mean() + 10 , line_text, color='red')

ax.set_facecolor('white')
plt.show()

The chart shows that most of the time the MVP get more salary, up to 800% than a average player. In avg. the MVP get 240% more salary than a average player.

# Conclusion

As the analyses shows the avg. salary of MBA players increased from about 50,000 in 1985 to about 4,500,000 in 2015. With an avg. increase rate about 8% and a big salary improvement from 1990 to 1991 with about 74%.

If a player mostly playes the LF position he could earn over the 30 years the most salary in average.

The last analyses shows that in avg. the MVP of an saison gets 240% more salary than the avg. of the players in this saison.

The analyses is limited that it just considers the regualr season games and not includes the playoff games. Also to calculate the average salary per position the portion of different positions a player plays would be a better fit for the analyses as using the position he plays most as basis.